In [1]:
import sys
import os

# moce two directories up and make that current directory
os.chdir("../..")
# print current directory
print(os.getcwd())

/Users/chinmayshrivastava/Documents/GitHub/kinara-rfp


In [2]:
from govrfp.rfpretriever.RFPRetriever import RFPRetriever
from llama_index.llms.openai import OpenAI

In [3]:
rfp = RFPRetriever(
    pdf_path="govrfp/rfpparser/RFP 2032H323 Amendment.pdf",
    verbose=True
)

100%|██████████| 146/146 [01:19<00:00,  1.83it/s]


{'_data': {'docstore/metadata': {'d610423c-9ff8-4f63-a3a5-ca0b108f429b': {'doc_hash': '04ba30a84253d75dd69da856c83ea2dc38ddfbca01cac9fd020c5d16ad930534'}, 'e394748f-26c3-42ea-8157-d515c3ce4330': {'doc_hash': 'a7968915bced1b08ee94d77295582fe99d3035ab1c834629086c5c1545736c99'}, '66a676ae-3b18-47a9-858e-3187cba6f5dc': {'doc_hash': '6fc3f0b381e1b1a89a4bb95ce8298cf8afb2bbb1b1813a1f8fa6c1b6b711dd7b'}, 'bc11c94d-43e6-487a-a938-f7647b092be2': {'doc_hash': '795086e210695c60551c25669f616fcdbd930d7c1984d95837a0c963a03d470b'}, '89120935-d331-4ea0-9f64-c1bd705b8d8c': {'doc_hash': 'bfb7301a617be9d6234a0625ac7255057940bf116557e164aff69580d1fc0f9c'}, '05e75b4b-aef9-4d12-acac-55d27a10e97e': {'doc_hash': '0dd8c633aaae25ca4bc8e281079a1f4222efdab13238488efdfcb02533d2a576'}, 'f279e3bc-7657-4f83-9ebe-86210fb27a46': {'doc_hash': 'ca769c72223be4d3a36f6bc1705c8f62c063a8c460ca7ad79878f0d8975f8da9'}, 'e747bf38-1fd2-43f4-b129-50bb4eb8bb27': {'doc_hash': '2975720f9474980a449310da27fcdf36c69688ec2ff7b5133886d313e16

In [4]:
rfp.annotations

{0: {'contains_date': <re.Match object; span=(504, 514), match='08/31/2023'>,
  'contains_currency': <re.Match object; span=(1217, 1220), match='$15'>,
  'contains_time': None,
  'contains_email': None,
  'contains_url': None,
  'contains_phone_number': None,
  'contains_evaluation_criteria': False,
  'contains_relevant_dates': True,
  'contains_communication_info': True},
 1: {'contains_date': <re.Match object; span=(729, 738), match='9/30/2023'>,
  'contains_currency': None,
  'contains_time': None,
  'contains_email': <re.Match object; span=(857, 886), match='Antoinette.Jones@treasury.gov'>,
  'contains_url': None,
  'contains_phone_number': <re.Match object; span=(844, 856), match='202-622-9318'>,
  'contains_evaluation_criteria': False,
  'contains_relevant_dates': True,
  'contains_communication_info': True},
 2: {'contains_date': None,
  'contains_currency': None,
  'contains_time': None,
  'contains_email': None,
  'contains_url': None,
  'contains_phone_number': None,
  'conta

In [7]:
# find chunks that contain evaluation criteria
eval_chunks = []
for chunk in rfp.annotations.keys():
    if rfp.annotations[chunk]["contains_evaluation_criteria"]:
        eval_chunks.append(chunk)
len(eval_chunks)

19

In [8]:
eval_chunks

[11,
 45,
 69,
 71,
 97,
 98,
 112,
 124,
 127,
 128,
 131,
 132,
 134,
 135,
 136,
 137,
 138,
 139,
 141]

In [19]:
llm = OpenAI(model="gpt-4")
PROMPT = (
    "For the given excerpt from an RFP, find the evaluation criteria and list them.\n"
    "Keem the criteria to a minimum by merging multiple criteria into one.\n"
    "If no evaluation criteria are present, return None.\n"
    "Excerpt:\n"
    "{excerpt}\n"
    "Evaluation Criteria:\n"
)

In [20]:
import tqdm
eval_criteria = []
for chunk in tqdm.tqdm(eval_chunks):
    prompt = PROMPT.format(excerpt=rfp.chunks()[chunk]["text"])
    response = llm.complete(prompt).text
    eval_criteria.append(response)

100%|██████████| 19/19 [01:06<00:00,  3.50s/it]


In [21]:
eval_criteria = [c for c in eval_criteria if "None" not in c]
eval_criteria

['1. Compatibility of ICT with accessibility-related software and assistive technology devices\n2. Adherence to the functional performance criteria as outlined in 36 CFR Chapter 3\n3. General performance as per Chapter 3: Functional Performance Criteria\n4. Compliance with standards referenced in Chapter 7\n5. Use of applications as per X 503\n6. Use of authoring tools as per X 504\n7. Incorporation by reference as per X 702\n8. Contractor performance as per IR 1052.242-9000 - Post Award Evaluation of Contractor Performance (Jun 2020)\n9. Preparation of interim and final evaluations in accordance with FAR 42.15\n10. Timely preparation of evaluations to coincide with the anniversary date of the contract.',
 '1. Exceeds 55% domestic content\n2. Contains a critical component in domestic end products\n3. Compliance with the policies and procedures of FAR part 25\n4. Compliance with the Buy American-Free Trade Agreements-Israeli Trade Act Certificate\n5. Certification of each end product as

In [23]:
#  save it into a file
with open("tests/mar-12/02-evaluation_criteria.txt", "w") as f:
    for c in eval_criteria:
        f.write(c + "\n")